In [8]:
# Author: Samuel Walters
# Date: 9/24/2024
# email: samuel.walters1@snhu.edu
# Description: This is the Original file for the CS-340 Milestone 6 project. This file will create a dashboard that will allow the user to filter through the data and display it in a table, pie chart, and map. 
# The user will be able to filter the data by the type of rescue the animal was trained for. The user will also be able to select a row in the table and the map will update to show the location of the animal.
# Original code was provided by SNHU and modified by Samuel Walters.
# Enhanced sections will be oulined as such.

# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
import base64
from dash.dependencies import Input, Output

# Testing imports
import time
import sys

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from Module6Milestone import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Data will be pulled from the MongoDB database
USER = "aacUser"
PASS = "SNHU1234"

shelter = AnimalShelter(USER, PASS)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
# Original df. This will be used when 
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = Dash('SimpleExample')

# Check here Image
image_name = 'GraziosoSalvareLogo.png'
# encoded_image = base64.b64encode(open(image_name, 'rb').read())

app.layout = html.Div([
    html.A([
        # html.Img(id='Logo',src='data:image/png;base64,{}'.format(encoded_image.decode())),
    ], href="snhu.edu"),
    html.Center(html.B(html.H1('Samuel Walters CS-340 Dashboard'))),
    html.Hr(),
    # Check here Radio buttons
    html.Div(
        # Add in radio buttons that will contain the different options
        dcc.RadioItems(
            id='filter_by_selection',
            options=[
                {'label': 'Water', 'value' : 'water_rescue'},
                {'label': 'Mountain / Wilderness', 'value' : 'mountain_wilderness'},
                {'label': 'Disaster / Tracking', 'value' : 'disaster_tracking'},
                {'label': 'Reset', 'value' : 'reset'},
            ],
            value = 'reset',
            labelStyle={'display': 'inline-flex'}
        )
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter_by_selection', 'value')])
def update_dashboard(filter_by_selection):
    start_time = time.time()
    # Will change based on which radio button. Used if statements as Switch is not in Python 3.9
    if filter_by_selection == 'water_rescue':
            df = pd.DataFrame(list(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            })))
    elif filter_by_selection == 'mountain_wilderness':
            df = pd.DataFrame(list(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            })))
    elif filter_by_selection == 'disaster_tracking':
            df = pd.DataFrame(list(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
            })))
        #resets the search to nothing to allow all results to be displayed
    elif filter_by_selection == 'reset':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    df.drop(columns=['_id'],inplace=True) # Drop _id column or will not display
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    end_time = time.time()  # End time measurement
    elapsed_time = end_time - start_time  # Calculate elapsed time
    print(f"update_dashboard callback execution time: {elapsed_time:.4f} seconds for", filter_by_selection)  # Log the execution time


    return (data,columns)

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names and breeds.
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData):
    #create the views
    if not viewData: #Default view will be the center of Austin Texas
        markerArray = (30.75,-97.48) #default marker at Austin City center
        toolTip = "Austin Texas"
        popUpHeading="Austin Texas"
        popUpParagraph="Austin Texas"

    else: # Update the marker view
        dff = pd.DataFrame(df.iloc[viewData]) #convert the datatable to pandas dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) # Get the latitude
        coordLong = float(dff['location_long'].to_string().split()[1]) # Get the longitude
        markerArray = (coordLat, coordLong) # Combine them
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    return [dl.Map(style={'width': '600px', 'height': '300px'}, center=[30.75,-97.48],
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
            ])
        ])
    ])
]


app.run_server(debug=True)    # Uncomment this line to enable debugging.
# app.run_server()                # Comment this line if enabling debugging

Connection Successful


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[8], line 192, in update_graphs(viewData=None)
    190 dff = pd.DataFrame.from_dict(viewData)
    191 #creates the values needed for the names and breeds.
--> 192 names = dff['breed'].value_counts().keys().tolist()
        dff = Empty DataFrame
Columns: []
Index: []
    193 values = dff['breed'].value_counts().tolist()
    194 #creates a pie chart based on the data above

File c:\Users\Samuel\source\repos\SNHU_CS_340\.venv\Lib\site-packages\pandas\core\frame.py:4102, in DataFrame.__getitem__(
    self=Empty DataFrame
Columns: []
Index: [],
    key='breed'
)
   4100 if self.columns.nlevels > 1:
   4101     return self._getitem_multilevel(key)
-> 4102 indexer = self.columns.get_loc(key)
        key = 'breed'
        self = Empty DataFrame
Columns: []
Index: []
   4103 if is_integer(indexer):
   4104     indexer = [indexer]

File c:

In [9]:
#Author: Samuel Walters
#Date: 9/24/2024
############### Enhancement 1 ###############
#Description:  This enhancement contains a cached copy of the each radio selection's dataframes that were filtered from the original data set. 
    # 1. This will allow the data to be displayed faster as the data will not need to be filtered each time the user selects a different radio button.
    # 2. This only makes API calls to login to the MongoDB database, and when the user selects the reset button.
    # 3. The dataframes do take up more memory.

# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
import base64
from dash.dependencies import Input, Output
import time


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from Module6Milestone import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Data will be pulled from the MongoDB database
USER = "aacUser"
PASS = "SNHU1234"

shelter = AnimalShelter(USER, PASS)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
# Original df. This will be used for the first read of the dataframe from MongoDB.
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

#######################
# Enhancement Section #
#######################
# Makes a copy of the dataframe that is filtered by the different types of rescues. This will allow the data to be filtered faster as the data will not need to be filtered each time the user selects a different
# radio button. This only makes API calls to login to the MongoDB database when the user selects the reset button.
# Results in O(1) time complexity for the data to be displayed, and memory complexity of O(n) as the dataframes are stored in memory.
df_Water = df[                          # DF for Water Rescue    
    (df['animal_type'] == 'Dog') &
    (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
    (df['sex_upon_outcome'] == 'Intact Female') &
    (df['age_upon_outcome_in_weeks'] >= 26) &
    (df['age_upon_outcome_in_weeks'] <= 156)
].copy()
# Dataframe copied then filtered from the original DF.
df_Mountain_Wilderness = df[            # DF for Mountain / Wilderness
                (df['animal_type'] == 'Dog') &
                (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
                (df['sex_upon_outcome'] == 'Intact Male') &
                (df['age_upon_outcome_in_weeks'] >= 26) &
                (df['age_upon_outcome_in_weeks'] <= 156)].copy()
# Dataframe copied then filtered from the original DF.
df_Disaster_Tracking = df[              # DF for Disaster / Tracking
                (df['animal_type'] == 'Dog') &
                (df['breed'].isin(['German Shepherd', 'Doberman Pinscher', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                (df['sex_upon_outcome'] == 'Intact Male') &
                (df['age_upon_outcome_in_weeks'] >= 20) &
                (df['age_upon_outcome_in_weeks'] <= 300)].copy()

#########################
# Dashboard Layout / View
#########################
app = Dash('SimpleExample')

# Check here Image
image_name = 'GraziosoSalvareLogo.png'
# encoded_image = base64.b64encode(open(image_name, 'rb').read())

app.layout = html.Div([
    html.A([
        # html.Img(id='Logo',src='data:image/png;base64,{}'.format(encoded_image.decode())),
    ], href="snhu.edu"),
    html.Center(html.B(html.H1('Samuel Walters CS-340 Dashboard'))),
    html.Hr(),
    # Check here Radio buttons
    html.Div(
        # Add in radio buttons that will contain the different options
        dcc.RadioItems(
            id='filter_by_selection',
            options=[
                {'label': 'Water', 'value' : 'water_rescue'},
                {'label': 'Mountain / Wilderness', 'value' : 'mountain_wilderness'},
                {'label': 'Disaster / Tracking', 'value' : 'disaster_tracking'},
                {'label': 'Reset', 'value' : 'reset'},
            ],
            value = 'reset',
            labelStyle={'display': 'inline-flex'}
        )
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter_by_selection', 'value')])
def update_dashboard(filter_by_selection):
    # Start time measurement
    start_time = time.time()
    # Will change based on which radio button. Used if statements as Switch is not in Python 3.9
    # The selection will then call the cached dataframes that were created from the original data set.
    # Provides near instant response as it has already been filtered.
    if filter_by_selection == 'water_rescue':
            df = df_Water
    elif filter_by_selection == 'mountain_wilderness':
            df = df_Mountain_Wilderness
    elif filter_by_selection == 'disaster_tracking':
            df = df_Disaster_Tracking
        # Reset the data. Makes new DB call
    elif filter_by_selection == 'reset':
        df = pd.DataFrame.from_records(shelter.read({}))
        
    if '_id' in df.columns:
        df.drop(columns=['_id'],inplace=True) # Drop _id column or will not display
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    end_time = time.time() # End time measurement
    elapsed_time = end_time - start_time
    print(f"update_map callback execution time: {elapsed_time:.4f} seconds for", filter_by_selection)  # Log the execution time

    data=df.to_dict('records')
        
    return (data,columns)

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names and breeds.
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData):
    #create the views
    if not viewData: #Default view will be the center of Austin Texas
        markerArray = (30.75,-97.48) #default marker at Austin City center
        toolTip = "Austin Texas"
        popUpHeading="Austin Texas"
        popUpParagraph="Austin Texas"

    else: # Update the marker view
        dff = pd.DataFrame(df.iloc[viewData]) #convert the datatable to pandas dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) # Get the latitude
        coordLong = float(dff['location_long'].to_string().split()[1]) # Get the longitude
        markerArray = (coordLat, coordLong) # Combine them
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    return [dl.Map(style={'width': '600px', 'height': '300px'}, center=[30.75,-97.48],
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
            ])
        ])
    ])
]


app.run_server(debug=True)    # Uncomment this line to enable debugging.
# app.run_server()                # Comment this line if enabling debugging

Connection Successful


update_map callback execution time: 0.1111 seconds for reset
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[9], line 192, in update_graphs(viewData=None)
    190 dff = pd.DataFrame.from_dict(viewData)
    191 #creates the values needed for the names and breeds.
--> 192 names = dff['breed'].value_counts().keys().tolist()
        dff = Empty DataFrame
Columns: []
Index: []
    193 values = dff['breed'].value_counts().tolist()
    194 #creates a pie chart based on the data above

File c:\Users\Samuel\source\repos\SNHU_CS_340\.venv\Lib\site-packages\pandas\core\frame.py:4102, in DataFrame.__getitem__(
    self=Empty DataFrame
Columns: []
Index: [],
    key='breed'
)
   4100 if self.columns.nlevels > 1:
   4101     return self._getitem_multilevel(key)
-> 4102 indexer = self.columns.get_loc(key)
        key = 'breed'
        self = Empty DataFrame
Columns: []
Index: []
   4103 if 

In [10]:
# Author: Samuel Walters
# Date: 9/24/2024
# email: samuel.walters1@snhu.edu
############### Enhancement 2 ###############
# Description: This enhancement contains the following changes:
        # 1. Changed the way the data is filtered to use the Pandas DataFrame instead of the MongoDB query. This will allow for faster filtering of the data without caching all the data as extra DataFrames.
        # 2. This experiment does NOT cache the dataframes that were filtered from the original data set.
        # 3. This experiment will only make API calls to login to the MongoDB database, and when the user selects the reset button.
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
import base64
from dash.dependencies import Input, Output
import time


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from Module6Milestone import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# Data will be pulled from the MongoDB database
USER = "aacUser"
PASS = "SNHU1234"

shelter = AnimalShelter(USER, PASS)


# This is the single dataframe that is called and read from the MongoDB database.
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = Dash('SimpleExample')

# Check here Image
image_name = 'GraziosoSalvareLogo.png'
# encoded_image = base64.b64encode(open(image_name, 'rb').read())

app.layout = html.Div([
    html.A([
        # html.Img(id='Logo',src='data:image/png;base64,{}'.format(encoded_image.decode())),
    ], href="snhu.edu"),
    html.Center(html.B(html.H1('Samuel Walters CS-340 Dashboard'))),
    html.Hr(),
    # Check here Radio buttons
    html.Div(
        # Add in radio buttons that will contain the different options
        dcc.RadioItems(
            id='filter_by_selection',
            options=[
                {'label': 'Water', 'value' : 'water_rescue'},
                {'label': 'Mountain / Wilderness', 'value' : 'mountain_wilderness'},
                {'label': 'Disaster / Tracking', 'value' : 'disaster_tracking'},
                {'label': 'Reset', 'value' : 'reset'},
            ],
            value = 'reset',
            labelStyle={'display': 'inline-flex'}
        )
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter_by_selection', 'value')])

# This function will filter the data based on the radio button selected. Enhanced to filter based on the original
# data set instead of making a new MongoDB call for every filter.
# This has a time complexity of O(n) as it will filter the data based on the original data set, and memory consumption of O(1) as it will not cache the data.
def update_dashboard(filter_by_selection):
    start_time = time.time()                            # Start time measurement
    # Will change based on which radio button. Used if statements as Switch is not in Python 3.9
    if filter_by_selection == 'water_rescue':
        df_filtered = df[
            (df['animal_type'] == 'Dog') &
            (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156)].copy()
    elif filter_by_selection == 'mountain_wilderness':
        df_filtered = df[
            (df['animal_type'] == 'Dog') &
            (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156)].copy()
    elif filter_by_selection == 'disaster_tracking':
        df_filtered = df[
                (df['animal_type'] == 'Dog') &
                (df['breed'].isin(['German Shepherd', 'Doberman Pinscher', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                (df['sex_upon_outcome'] == 'Intact Male') &
                (df['age_upon_outcome_in_weeks'] >= 20) &
                (df['age_upon_outcome_in_weeks'] <= 300)].copy()
    # Reset the data to the original data set, makes new DB call for any added data.
    elif filter_by_selection == 'reset':
        df_filtered = pd.DataFrame.from_records(shelter.read({}))
        
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'],inplace=True) # Drop _id column or will not display
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"update_map callback execution time: {elapsed_time:.4f} seconds for", filter_by_selection)  # Log the execution time

    data=df_filtered.to_dict('records')
        
    return (data,columns)

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names and breeds.
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData):
    #create the views
    if not viewData: #Default view will be the center of Austin Texas
        markerArray = (30.75,-97.48) #default marker at Austin City center
        toolTip = "Austin Texas"
        popUpHeading="Austin Texas"
        popUpParagraph="Austin Texas"

    else: # Update the marker view
        dff = pd.DataFrame(df.iloc[viewData]) #convert the datatable to pandas dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) # Get the latitude
        coordLong = float(dff['location_long'].to_string().split()[1]) # Get the longitude
        markerArray = (coordLat, coordLong) # Combine them
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    return [dl.Map(style={'width': '600px', 'height': '300px'}, center=[30.75,-97.48],
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
            ])
        ])
    ])
]


app.run_server(debug=True)    # Uncomment this line to enable debugging.
# app.run_server()                # Comment this line if enabling debugging

Connection Successful


update_map callback execution time: 0.1215 seconds for reset
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[10], line 183, in update_graphs(viewData=None)
    181 dff = pd.DataFrame.from_dict(viewData)
    182 #creates the values needed for the names and breeds.
--> 183 names = dff['breed'].value_counts().keys().tolist()
        dff = Empty DataFrame
Columns: []
Index: []
    184 values = dff['breed'].value_counts().tolist()
    185 #creates a pie chart based on the data above

File c:\Users\Samuel\source\repos\SNHU_CS_340\.venv\Lib\site-packages\pandas\core\frame.py:4102, in DataFrame.__getitem__(
    self=Empty DataFrame
Columns: []
Index: [],
    key='breed'
)
   4100 if self.columns.nlevels > 1:
   4101     return self._getitem_multilevel(key)
-> 4102 indexer = self.columns.get_loc(key)
        key = 'breed'
        self = Empty DataFrame
Columns: []
Index: []
   4103 if